In [1]:
import networkx as nx
import numpy as np
import kaiwu as kw
import neal
import dimod
import gurobipy as gp
from gurobipy import GRB

2025-04-28 20:38:51,371 - numexpr.utils - _init_num_threads - 148 - INFO - Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2025-04-28 20:38:51,371 - numexpr.utils - _init_num_threads - 160 - INFO - NumExpr defaulting to 8 threads.


ModuleNotFoundError: No module named 'neal'

: 

In [7]:
def generate_regular_graph(d,n):
    """生成连通的n节点d-正则图"""
    while True:
        G = nx.random_regular_graph(d,n,seed=1)
        ising_matrix = nx.to_numpy_array(G, dtype=int)
        if nx.is_connected(G):
            return G,ising_matrix

In [8]:
def qubo_matrix_to_dict(matrix):
    qubo_dict = {}
    rows = len(matrix)
    cols = len(matrix[0])

    for i in range(rows):
        for j in range(cols):
            if matrix[i][j] != 0:
                qubo_dict[(i, j)] = matrix[i][j]
    return qubo_dict

def ising_matrix_to_dict(ising_matrix):
    n = len(ising_matrix)
    h = {}
    J = {}

    for i in range(n):
        # 提取对角元素作为 h
        h[i] = ising_matrix[i][i]

        for j in range(i + 1, n):
            # 提取非对角元素作为 J
            if ising_matrix[i][j] != 0:
                J[(i, j)] = ising_matrix[i][j]
    return h, J

def qubo_to_matrix(qubo):
    n = len(set(i for i, j in qubo.keys()))
    qubo_matrix = np.zeros((n,n))
    for k,v in qubo.items():
        c_index = int(k[0])
        r_index = int(k[1])
        qubo_matrix[c_index,r_index] = v
    return qubo_matrix

def calculate_cut(assignment, J):
    cut = 0
    for (i, j), value in J.items():
        if assignment[i] != assignment[j]:
            cut += abs(value)
    return cut

In [9]:
def calculate_sa_running_time(response):
    preprocessing_ns = response.info['timing']['preprocessing_ns'] / 1e6
    sampling_ns = response.info['timing']['sampling_ns'] / 1e6
    postprocessing_ns = response.info['timing']['postprocessing_ns'] / 1e6
    total_time = preprocessing_ns+sampling_ns+postprocessing_ns
    return total_time

def sa_compute(G,h,J):
    solver = neal.SimulatedAnnealingSampler()
    response = solver.sample_ising(h,J,num_reads=100)
    min_energy = response.first.energy
    min_energy_samples = response.filter(lambda s: s.energy == min_energy)  
    max_cut = 0
    for sample in min_energy_samples:
        assignment = dict(sample)
        cut = calculate_cut(assignment, J)
        if cut > max_cut:
            max_cut = cut
    running_time = calculate_sa_running_time(response)
    print("模拟退火求解的最大割数:", max_cut)
    print(f"模拟退火运行时间:{running_time:.2f}毫秒")
    return max_cut,running_time

In [10]:
def solve_maxcut_ising(h,J):
    # h, J = ising_matrix_to_dict(ising_matrix)
    n = len(h)

    # 创建模型
    model = gp.Model("MaxCut_Ising")
    model.setParam("OutputFlag", 0) 
    model.setParam('TimeLimit', 600)
    
    # 创建变量
    x = model.addVars(n, vtype=GRB.BINARY, name="x")

    # 目标函数
    objective = gp.QuadExpr()
    for i in range(n):
        objective += h[i] * (2 * x[i] - 1)
    for (i, j), value in J.items():
        objective += value * (2 * x[i] - 1) * (2 * x[j] - 1)

    model.setObjective(objective, GRB.MINIMIZE)
    model.optimize()
    runtime = model.Runtime
    
    if model.status == GRB.OPTIMAL:
        solution = [x[i].x for i in range(n)]
        print("找到了最优解！")
    elif model.status == GRB.TIME_LIMIT:
        if model.getAttr(GRB.Attr.SolCount) > 0:
            solution = [x[i].x for i in range(n)]
            print("求解时间超过上限，返回当前最优解。")
        else:
            solution = None
            print("求解时间超过上限，未找到可行解。")
    else:
        solution = None
        print("未找到最优解，求解状态码:", model.status)

    if solution is not None:
        obj_val = model.objVal
    else:
        obj_val = None

    return solution, obj_val, runtime

# 模拟退火

In [6]:
n_list = np.arange(100,1100,100)
d = 3
for n in n_list:
    print("-"*50)
    print(f"n={n} 的{d}-正则图 模拟退火求解测试")
    G,ising_matrix = generate_regular_graph(d,n)
    h,J = ising_matrix_to_dict(ising_matrix)
    max_cut,running_time = sa_compute(G,h,J)

--------------------------------------------------
n=100 的3-正则图 模拟退火求解测试
模拟退火求解的最大割数: 138
模拟退火运行时间:93.61毫秒
--------------------------------------------------
n=200 的3-正则图 模拟退火求解测试
模拟退火求解的最大割数: 272
模拟退火运行时间:177.23毫秒
--------------------------------------------------
n=300 的3-正则图 模拟退火求解测试
模拟退火求解的最大割数: 412
模拟退火运行时间:259.20毫秒
--------------------------------------------------
n=400 的3-正则图 模拟退火求解测试
模拟退火求解的最大割数: 552
模拟退火运行时间:342.60毫秒
--------------------------------------------------
n=500 的3-正则图 模拟退火求解测试
模拟退火求解的最大割数: 689
模拟退火运行时间:424.48毫秒
--------------------------------------------------
n=600 的3-正则图 模拟退火求解测试
模拟退火求解的最大割数: 825
模拟退火运行时间:507.13毫秒
--------------------------------------------------
n=700 的3-正则图 模拟退火求解测试
模拟退火求解的最大割数: 965
模拟退火运行时间:590.16毫秒
--------------------------------------------------
n=800 的3-正则图 模拟退火求解测试
模拟退火求解的最大割数: 1104
模拟退火运行时间:691.10毫秒
--------------------------------------------------
n=900 的3-正则图 模拟退火求解测试
模拟退火求解的最大割数: 1242
模拟退火运行时间:761.12毫秒
---------------------------

# Gurobi

In [11]:
n_list = np.arange(100,1100,100)
d = 3
for n in n_list:
    print("-"*50)
    print(f"n={n} 的{d}-正则图 Gurobi 求解测试")
    G,ising_matrix = generate_regular_graph(d,n)
    h,J = ising_matrix_to_dict(ising_matrix)
    solution,energy,runtime = solve_maxcut_ising(h,J)
    n_cut = calculate_cut(solution,J)
    print("Gurobi 求解的最大割数:",n_cut)
    print(f"Gurobi 求解用时: {runtime:.2f} 秒")

--------------------------------------------------
n=100 的3-正则图 Gurobi 求解测试
Set parameter LicenseID to value 2600822
找到了最优解！
Gurobi 求解的最大割数: 138
Gurobi 求解用时: 0.04 秒
--------------------------------------------------
n=200 的3-正则图 Gurobi 求解测试
找到了最优解！
Gurobi 求解的最大割数: 272
Gurobi 求解用时: 1.71 秒
--------------------------------------------------
n=300 的3-正则图 Gurobi 求解测试
找到了最优解！
Gurobi 求解的最大割数: 412
Gurobi 求解用时: 3.11 秒
--------------------------------------------------
n=400 的3-正则图 Gurobi 求解测试
找到了最优解！
Gurobi 求解的最大割数: 552
Gurobi 求解用时: 10.25 秒
--------------------------------------------------
n=500 的3-正则图 Gurobi 求解测试
找到了最优解！
Gurobi 求解的最大割数: 690
Gurobi 求解用时: 99.17 秒
--------------------------------------------------
n=600 的3-正则图 Gurobi 求解测试
找到了最优解！
Gurobi 求解的最大割数: 827
Gurobi 求解用时: 402.16 秒
--------------------------------------------------
n=700 的3-正则图 Gurobi 求解测试
求解时间超过上限，返回当前最优解。
Gurobi 求解的最大割数: 966
Gurobi 求解用时: 600.03 秒
--------------------------------------------------
n=800 的3-正则图 Gurobi 求解测试

# 环状图

In [180]:
n_list = np.arange(100,1100,100)
for n in n_list:
    print("-"*50)
    print(f"n={n} 的环状图 Gurobi 求解测试")
    G = nx.cycle_graph(n)
    ising_matrix = nx.to_numpy_array(G, dtype=int)
    h,J = ising_matrix_to_dict(ising_matrix)
    solution,energy,runtime = solve_maxcut_ising(h,J)
    n_cut = calculate_cut(solution,J)
    print("Gurobi 求解的最大割数:",n_cut)
    print(f"Gurobi 求解用时: {runtime:.2f} 秒")

--------------------------------------------------
n=100 的环状图 Gurobi 求解测试
Gurobi 求解的最大割数: 100
Gurobi 求解用时: 0.00 秒
--------------------------------------------------
n=200 的环状图 Gurobi 求解测试
Gurobi 求解的最大割数: 200
Gurobi 求解用时: 0.00 秒
--------------------------------------------------
n=300 的环状图 Gurobi 求解测试


GurobiError: Model too large for size-limited license; visit https://gurobi.com/unrestricted for more information